In [1]:
# Soft Prompt Tuning :: Debug 

from optm.soft_prompt import *

model, tokenizer = load_hf_model_precise("Qwen/Qwen2.5-0.5B-Instruct") # load model & tokenizer

tf_dataset = ClsCommentDataset(load_tf_data, tokenizer, train=True)

# initializing soft-prompt model 
model_with_soft_prompt = SoftPromptLLM(model, tokenizer, 3, initialize_from_vocab=True)

# Usage
num_epochs = 5
learning_rate = 1e-3
train_dataloader = DataLoader(tf_dataset, batch_size=24, shuffle=True)

In [9]:
# prepare test data 
# testset = ClsCommentDataset(load_tf_data, tokenizer, train=False)

def test_model(model_with_soft_prompt, tokenizer, test_data, batch_size=24):

    batch_size = 24 

    data_len = len(test_data["prompt"])

    scores = []

    from tqdm import tqdm
    # add progress bar 
    for i in tqdm(range(0, data_len, batch_size), desc="Testing model"):
        prompts = test_data["prompt"][i:i+batch_size]
        labels = test_data["label"][i:i+batch_size]
        comments = test_data["comment"][i:i+batch_size]
        
        query_prompts = []
        for prompt, comment, label in zip(prompts, comments, labels):
            query_prompt, response_prompt = format_prompt_instruction_tuned(prompt, comment, label, tokenizer, previous_messages = [])
            query_prompts.append(query_prompt)
            
        generated_responses = model_with_soft_prompt.generate(query_prompts)
        
        for response, label in zip(generated_responses, labels):
            score = label_metric(response, label)
            scores.append(score)

    avg_score = sum(scores) / len(scores)
    print(f"Average score: {avg_score}")
    
    return avg_score

In [5]:
# model_with_soft_prompt.generate("hello")

In [ ]:
trained_model = train_soft_prompt(
    model_with_soft_prompt,
    train_dataloader,
    num_epochs=num_epochs,
    learning_rate=learning_rate,
    accumulation_steps=4
)

In [5]:
# Test model performance after training
from optm.soft_prompt import *

train_data, test_data = load_tf_data()

idx = 0 
# training data not properly formatted here
prompt, response = format_prompt_instruction_tuned(test_data["prompt"][idx], test_data["comment"][idx], test_data["label"][idx], tokenizer, previous_messages = [])

In [10]:
# Build a STaR data curation pipeline 

# - Use available model to generate diverse response to the current prompt
# - Filter out repeatitive responses 
# - the parsing functional is implemented into Node already, let's port to that before we do STaR -- same approach essentially here 

# def parse_response(response: str) -> dict: 
    